# S2HC_ETBFcalc

**Date:** <br>
10 June 2022 <br>
**Background:** <br>
Issue -  https://github.com/Thomas-Moore-Creative/NCI-ACCESS-S2-ARD/issues/7 <br>
**Author(s):**<br>
Thomas Moore<br>

### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,processes=1,memory="47GB",walltime='02:00:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.136:45005,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# load main collections from zarr

In [6]:
path = '/g/data/xv83/users/tm4888/data/ACCESS-S2/hindcast/'

In [8]:
%%time
ds_TUV = xr.open_zarr(path+'accessS2.HC.ocean.masked.AUSWCPregion.TUV_025deg.zarr',consolidated=True)

CPU times: user 134 ms, sys: 30.9 ms, total: 165 ms
Wall time: 434 ms


# Grid information for ACCESS S2?

In [9]:
%%time
ds_S2_Tgrid = xr.open_mfdataset('/g/data/ux62/access-s2/reanalysis/ocean/temp/mo_temp_1990.nc')

CPU times: user 90.7 ms, sys: 21.4 ms, total: 112 ms
Wall time: 863 ms


## depth weights

In [10]:
depth_weights = ds_S2_Tgrid.deptht_bounds.diff(dim='ndepth_bounds').squeeze().rename({'deptht':'depth'})
depth_weights.values

array([  1.0239066,   1.0792834,   1.1481192,   1.2337434,   1.3401852,
         1.4722052,   1.6352429,   1.8352118,   2.0780945,   2.369277 ,
         2.712654 ,   3.1096478,   3.5584164,   4.0536633,   4.5873146,
         5.150135 ,   5.7339096,   6.333458 ,   6.948097 ,   7.582039 ,
         8.244095 ,   8.946861 ,   9.705849 ,  10.538765 ,  11.465065 ,
        12.505692 ,  13.683044 ,  15.021072 ,  16.545197 ,  18.282547 ,
        20.261673 ,  22.512512 ,  25.065903 ,  27.952911 ,  31.204071 ,
        34.848145 ,  38.910614 ,  43.411926 ,  48.365784 ,  53.776886 ,
        59.638855 ,  65.93317  ,  72.62689  ,  79.67352  ,  87.01233  ,
        94.57056  , 102.265564 , 110.00891  , 117.70947  , 125.277954 ,
       132.63196  , 139.69739  , 146.41333  , 152.73181  , 158.6195   ,
       164.05664  , 169.03613  , 173.56226  , 177.64746  , 181.31226  ,
       184.5813   , 187.48242  , 190.04639  , 192.30273  , 194.28223  ,
       196.01294  , 197.52246  , 198.83594  , 199.97656  , 200.9

In [11]:
depth_weights

<xarray.DataArray 'deptht_bounds' (depth: 75)>
dask.array<getitem, shape=(75,), dtype=float32, chunksize=(75,), chunktype=numpy.ndarray>
Coordinates:
  * depth    (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03

# $\big\Downarrow$ Calculate

# EKE

In [12]:
def calc_EKE(u, v, time_name = 'time'):
    '''
    Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}
    u,v are x and y currents as an xarray data array
    '''
    u_mean = u.mean(time_name)
    v_mean = v.mean(time_name)
    MKE = 0.5*(u_mean**2 + v_mean**2).rename('MKE') # currents
    EKE = ( 0.5 * ((u-u_mean)**2 + (v-v_mean)**2) ).rename('EKE') # eddies
    return EKE, MKE

In [14]:
%%time
[EKE, MKE] = calc_EKE(ds_TUV.u, ds_TUV.v, time_name='lead_time')

CPU times: user 68.7 ms, sys: 4.46 ms, total: 73.1 ms
Wall time: 77.8 ms


# U100

In [15]:
u100 = ds_TUV.u.sel(depth=100,method='nearest')

# V100

In [16]:
v100 = ds_TUV.v.sel(depth=100,method='nearest')

# u100_300

## need to use depth weighted mean

In [17]:
def weighted_ocean_mean(ds,weights=None,dim=None,sel_dict=None):
    """
    weighted_ocean_mean
    Returns: ds_weighted
    Defaults: no spatial and time selection, no weights, no selected dims
    Author: Thomas Moore (based on Dougie Squire code)
    Date created: 11/02/2020

    Assumptions:
    Dataset or Dataarray = ds
    Use:
    Limitations:
    """

    if weights is None:
        return ds.sel(sel_dict).mean(dim)
    else:
        #weights = (0 * da + 1) * weights
        #return (da * weights).sum(dim) / weights.sum(dim)
        return (ds.sel(sel_dict) * weights).sum(dim,skipna=True) / weights.where(ds.sel(sel_dict).notnull()).sum(dim,skipna=True)

In [18]:
u100_300 = helper.weighted_ocean_mean(ds_TUV.u,weights=depth_weights,dim='depth',sel_dict={'depth':slice(100,300)})

# v100_300

## need to use depth weighted mean

In [19]:
v100_300 = helper.weighted_ocean_mean(ds_TUV.v,weights=depth_weights,dim='depth',sel_dict={'depth':slice(100,300)})

# EKE_300

## need to account for variable cell thickness without formal integration

In [20]:
eke300 = (EKE * depth_weights).sel(depth=slice(0,300)).sum('depth',skipna=True)
eke300 = eke300.where(eke300 != 0)

# EKE_2000

## need to account for variable cell thickness without formal integration

In [21]:
eke2000 = (EKE * depth_weights).sel(depth=slice(0,2000)).sum('depth',skipna=True)
eke2000 = eke2000.where(eke2000 != 0)

# Merge added calculated variables

In [22]:
ds_TUV['u100'] = u100
ds_TUV['v100'] = v100
ds_TUV['u100_300'] = u100_300
ds_TUV['v100_300'] = v100_300
ds_TUV['eke300'] = eke300
ds_TUV['eke2000'] = eke2000

# Drop 3D u & v as well as depth coordinate

In [23]:
ds_TUV = ds_TUV.drop_vars(['u','v'])
ds_TUV = ds_TUV.drop_dims(['depth'])

In [24]:
ds_TUV.nbytes/1e9

0.07080512

In [25]:
ds_TUV

<xarray.Dataset>
Dimensions:              (lat: 361, lon: 721, lead_time: 3, ncorners: 4)
Coordinates:
    forecast_start_date  datetime64[ns] ...
  * lat                  (lat) float64 -70.0 -69.75 -69.5 ... 19.5 19.75 20.0
  * lead_time            (lead_time) datetime64[ns] 2018-03-16T12:00:00 ... 2...
  * lon                  (lon) float64 90.0 90.25 90.5 ... 269.5 269.8 270.0
Dimensions without coordinates: ncorners
Data variables: (12/25)
    areat                (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    areau                (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    areav                (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    hc300                (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    latt_bounds          (ncorners, lat, lon) float32 dask.array<chunksize=(4, 361, 721), meta=np.ndarray>
    latu_bounds          (ncorners, lat, lon) float32 dask.array<chunksize=(4, 361, 721), meta=np.ndarray>
    ...                   ...
    u100                 (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    v100                 (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    u100_300             (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    v100_300             (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    eke300               (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
    eke2000              (lead_time, lat, lon) float32 dask.array<chunksize=(3, 361, 721), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

# Write out single zarr collection

In [26]:
%%time
ds_TUV.to_zarr(path+'accessS2.HC.ocean.masked.AUSWCPregion.ETBFvars.zarr',consolidated=True)

CPU times: user 3.54 s, sys: 492 ms, total: 4.03 s
Wall time: 19.1 s


# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [ ]:
client.restart()

In [27]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [ ]:
client.restart()